In [ ]:
import fev

c:\Users\Luca\miniconda3\envs\FairTSBench\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Tutorial 1

All tutorial can be accessed here https://github.com/autogluon/fev?tab=readme-ov-file

In [ ]:
# Create a task from a dataset stored on Hugging Face Hub
task = fev.Task(
    dataset_path="autogluon/chronos_datasets",
    dataset_config="ercot",
    horizon=24,
    num_windows=2,
)

In [ ]:
# A task consists of multiple rolling evaluation windows
for window in task.iter_windows():
    print(window)

In [ ]:
# Load data available as input to the forecasting model
past_data, future_data = task.get_window(0).get_input_data()

In [ ]:
# past data before the forecast horizon.
past_data

In [ ]:
past_data[0]

In [ ]:
# future data that is known at prediction time (item ID, future timestamps, static and known covariates)
future_data

In [ ]:
future_data[0]

In [ ]:
import numpy as np


def naive_forecast(y: list, horizon: int) -> dict[str, list]:
    # Make predictions for a single time series
    return {"predictions": [y[np.isfinite(y)][-1] for _ in range(horizon)]}

predictions_per_window = []
for window in task.iter_windows():
    past_data, future_data = window.get_input_data()
    predictions = [
        naive_forecast(ts[task.target], task.horizon) for ts in past_data
    ]
    predictions_per_window.append(predictions)

In [ ]:
eval_summary = task.evaluation_summary(predictions_per_window, model_name="naive")
eval_summary

In [ ]:
import pandas as pd

summaries = pd.read_csv("https://raw.githubusercontent.com/autogluon/fev/refs/heads/main/benchmarks/example/results/results.csv")
summaries.head()

In [ ]:
# Evaluation summaries can be provided as dataframes, dicts, JSON or CSV files
fev.leaderboard(summaries, baseline_model="seasonal_naive")

# Tutorial 3

In [1]:
import warnings
from pathlib import Path

import datasets
import numpy as np
from tqdm.auto import tqdm

import fev

warnings.simplefilter("ignore")
datasets.disable_progress_bars()

c:\Users\Luca\miniconda3\envs\FairTSBench\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
# Create a toy dataset with a single time series
ts = {
    "id": "A",
    "timestamp": pd.date_range("2025-01-01", freq="D", periods=10),
    "target": list(range(10)),
}
ds = datasets.Dataset.from_list([ts])
dataset_path = "/tmp/toy_dataset.parquet"
ds.to_parquet(dataset_path);

In [4]:
task = fev.Task(
    dataset_path=dataset_path,
    horizon=3,
    num_windows=2,
)

# Show the original dataset before any splits (for reference only)
full_dataset = task.load_full_dataset()
print(full_dataset)
print(full_dataset[0])

: 

In [ ]:
# Show how data is split across the 2 evaluation windows
for window_index, window in enumerate(task.iter_windows()):
    past, future = window.get_input_data()
    ground_truth = window.get_ground_truth()
    print(f"Window {window_index} (cutoff={window.cutoff}):")
    print(f"  Past data:    {past[0]['target']}")
    print(f"  Ground truth: {ground_truth[0]['target']}")

In [ ]:
# Example 1: Start evaluation earlier with initial_cutoff
task = fev.Task(
    dataset_path=dataset_path,
    horizon=3,
    num_windows=2,
    initial_cutoff=-8,
)

for window_index, window in enumerate(task.iter_windows()):
    past, future = window.get_input_data()
    ground_truth = window.get_ground_truth()
    print(f"Window {window_index} (cutoff={window.cutoff}):")
    print(f"  Past data:    {past[0]['target']}")
    print(f"  Ground truth: {ground_truth[0]['target']}")

In [ ]:
# Example 3: Use pandas timestamp-like strings
task = fev.Task(
    dataset_path=dataset_path,
    horizon=3,
    num_windows=2,
    initial_cutoff="2025-01-05",
    window_step_size="2D",
)

for window_index, window in enumerate(task.iter_windows()):
    past, future = window.get_input_data()
    ground_truth = window.get_ground_truth()
    print(f"Window {window_index} (cutoff={window.cutoff}):")
    print(f"  Past data:    {past[0]['target']}")
    print(f"  Ground truth: {ground_truth[0]['target']}")

In [ ]:
task = fev.Task(
    dataset_path="autogluon/chronos_datasets",
    dataset_config="m4_hourly",
    horizon=24,
    num_windows=2,
)

In [ ]:
task.predictions_schema

In [ ]:
def naive_forecast(window: fev.EvaluationWindow) -> datasets.Dataset:
    assert len(window.target_columns) == 1, "only univariate forecasting supported"
    predictions: list[dict[str, np.ndarray]] = []
    past_data, future_data = window.get_input_data()
    for ts in past_data:
        y = ts[window.target_columns[0]]
        predictions.append(
            {"predictions": np.array([y[-1] for _ in range(window.horizon)])}
        )
    return datasets.Dataset.from_list(predictions)

window = task.get_window(0)
predictions = naive_forecast(window)
predictions

In [ ]:
print(predictions[0])

In [ ]:
predictions_per_window = [naive_forecast(window) for window in task.iter_windows()]
task.evaluation_summary(predictions_per_window, model_name="naive")

multivariate foecasting

In [ ]:
task = fev.Task(
    dataset_path="autogluon/fev_datasets",
    dataset_config="ETT_1H",
    horizon=3,
    target=["OT", "LUFL", "LULL"],
)

In [ ]:
past_data, future_data = task.get_window(0).get_input_data()
print(past_data)
print(future_data)

In [ ]:
def naive_forecast_multivariate(window: fev.EvaluationWindow) -> datasets.DatasetDict:
    """Predicts the last observed value in each multivariate column."""
    past_data, future_data = window.get_input_data()
    predictions = datasets.DatasetDict()
    for col in window.target_columns:
        predictions_for_column = []
        for ts in past_data:
            predictions_for_column.append({"predictions": [ts[col][-1] for _ in range(window.horizon)]})
        predictions[col] = datasets.Dataset.from_list(predictions_for_column)
    return predictions

In [ ]:
window = task.get_window(0)
predictions_per_window = naive_forecast_multivariate(window).cast(task.predictions_schema)
predictions_per_window

In [ ]:
for col in task.target_columns:
    print(f"Predictions for column '{col}'")
    print(f"\t{predictions_per_window[col].to_list()}")

In [ ]:
task.evaluation_summary([predictions_per_window], model_name="naive")

In [ ]:
task = fev.Task(
    dataset_path="autogluon/fev_datasets",
    dataset_config="ETT_1H",
    horizon=3,
    generate_univariate_targets_from=["OT", "LUFL", "LULL"],
    target="target",  # new name for the target columns ['OT', 'LUFL', 'LULL'] after splitting
)

In [ ]:
past_data, future_data = task.get_window(0).get_input_data()
print(past_data)
print(future_data)

In [ ]:
past_data["id"]

In [ ]:
def naive_forecast_univariate(window: fev.EvaluationWindow) -> datasets.Dataset:
    """Predicts the last observed value."""
    past_data, future_data = window.get_input_data()
    predictions = []
    for ts in past_data:
        predictions.append({"predictions": [ts[window.target_columns[0]][-1] for _ in range(window.horizon)]})
    return datasets.Dataset.from_list(predictions)

In [ ]:
predictions_per_window = []
for window in task.iter_windows():
    predictions_per_window.append(naive_forecast_univariate(window))
task.evaluation_summary(predictions_per_window, model_name="naive")

Evaluating on a benchmark (see tutorial)